In [ ]:
# instalando pacotes
# %pip install sklearn
# %pip install os
# %pip install xgboost

In [ ]:
# importando as bibliotecas para leitura dos dados e criação de gráficos
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import sklearn

# configurando pandas para mostrar todas as linhas e colunas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None) 

# configurando pandas para não mostrar notação científica para números
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
# Ler os dados do banco.
df_clientes = pd.read_csv('data\Base_CadastroClientes_analise.csv', sep=';')
df_clientes.shape

Cada linha é um cliente, portanto vamos verificar se isso é verdade?

In [ ]:
df_clientes['id'].is_unique

In [ ]:
# Vamos transformar id em indice do dataframe
df_clientes = df_clientes.set_index('id').sort_values(by= 'id', ascending=True)
df_clientes.head()

In [ ]:
# Avaliando os tipos
df_clientes.dtypes

In [ ]:
# Transforma varíaveis com data em formato datetime
df_clientes['dataNascimento'] = pd.to_datetime(df_clientes['dataNascimento'], format = '%d/%m/%Y')
df_clientes['dataCadastro'] = pd.to_datetime(df_clientes['dataCadastro'], format = '%d/%m/%Y')

In [ ]:
df_clientes['dataNascimento'].head()


In [ ]:
df_clientes.info()

## <a> Começando com estatística descritiva </a>

Conhecer bem as medidas estatísticas, de tendência central, dispersão, separatrizes, distribuições, é essencial para conhecermos melhor os dados em que estamos trabalhando. Qual a distribuição de tempo de relacionamento? Da valor do contrato? Da renda mensal? A base está desbalanceada?

In [ ]:
df_clientes.describe()

In [ ]:
# Analisando a distribuição de tempo de relacionamento (meses na empresa)
sns.set_style("darkgrid")
plt.tight_layout()
plt.figure(figsize = (15,8))

sns.histplot(data=df_clientes, x='range_tempoRelacionamento' )

In [ ]:
# Mudando o padrão de quartis para decis. linspace divide em espaços iguais um intervalo de números (0 a 1 com 11 intervalos)
df_clientes.describe(percentiles=np.linspace(0, 1, 11))

In [ ]:
# Analisando a distribuição do cidadeRendaPercapita
sns.set_style("darkgrid")
plt.tight_layout()
plt.figure(figsize = (15,8))

sns.histplot(data=df_clientes, x='range_cidadeRendaPercapita')

In [ ]:
# Analisando a distribuição da estadoRendaPercapita
sns.set_style("darkgrid")
plt.tight_layout()
plt.figure(figsize = (15,8))

sns.histplot(data=df_clientes, x='range_estadoRendaPercapita')

In [ ]:
# Analisando a distribuição da estadoRendaPercapita
sns.set_style("darkgrid")
plt.tight_layout()
plt.figure(figsize = (15,8))

sns.histplot(data=df_clientes, x='range_diasAtraso')

In [ ]:
len(df_clientes.loc[df_clientes['range_diasAtraso'] == 'NO PRAZO']), len(df_clientes.loc[df_clientes['range_diasAtraso'] != 'NO PRAZO'])

In [ ]:
# Se o modelo chutar tudo como "atrasado"
100 * len(df_clientes.loc[df_clientes['range_diasAtraso'] != 'NO PRAZO']) / df_clientes.shape[0]

Ou seja, acurácia perto de 59.34 quer dizer que usamos machine learning pra nada :D

## <a> Relação entre as features e a variável target </a>

Uma análise interessante é avaliar relações entre as variáveis preditoras com a target. Vamos analisar as dispersões das variáveis preditoras com o churn

In [ ]:
# O pairplot faz gráficos de dispersão para os pares de variáveis (incluindo a target)
# Na diagonal principal ele mostra o histograma
# sns.pairplot(data=df_clientes)

### Apesar de já ter visto o pairplot uma vez realizei o comentário 
# para que o código fosse executado mais rápido

## <a> Codificação de Variáveis Categóricas </a>

Lembrando que os modelos de machine learning não sabem o que são categorias em sua maioria, devemos, portanto, codificar as variáveis de sexo, parceiro(a), dependentes, tipo de cobrança e todos os tipos de serviço.

In [ ]:
df_clientes.head()

Vamos analisar quantas classes possuem as variáveis categóricas para saber como codificar cada uma

In [ ]:
df_clientes._get_numeric_data().columns

In [ ]:
colunas_categoricas = [coluna for coluna in df_clientes.columns if coluna not in df_clientes._get_numeric_data().columns]
colunas_categoricas

In [ ]:
# Mapear sexo
df_clientes['sexo'] = df_clientes['sexo'].map({'Feminino': 1, 'Masculino': 0})

# Mapear diasAtraso - variável target para o modelo
df_clientes['status'] = df_clientes['diasAtraso'].apply( lambda x: 0 if x == 0 else 1  )

In [ ]:
# Será que funcionou?

display(df_clientes['sexo'].value_counts())
display(df_clientes['status'].value_counts())

In [ ]:
df_clientes.loc[:, ['sexo']].isnull().sum()

In [ ]:
for coluna_categorica in colunas_categoricas:
    display(df_clientes[coluna_categorica].value_counts())

In [ ]:
#Remover duplicadas
df_clientes = df_clientes.drop_duplicates(subset=['cpf'], keep='first')
df_clientes = df_clientes.drop_duplicates(subset=['email'], keep='first')

#Remover variáveis categoricas com muitas entradas ou com informações duplicadas
df_clientes = df_clientes.drop(['cep', 'cidade','dataNascimento','dataCadastro',
                                'profissao','empresa','veiculo','nome','estado',
                                'rendaMensal', 'valorContrato', 'endereco', 
                                'diasAtraso','longitude', 'latitude', 'telefone',
                                'email', 'cpf', 'tipoSanguineo'], axis=1)
# Mapear variáveis com Range
df_clientes['range_valorContrato'] = df_clientes['range_valorContrato'].apply( lambda x: 1 if x == '1 - DE R$ 10.000 A R$ 20.000' else
                                                                      2 if x == '2 - DE R$ 20.001 A R$ 40.000' else
                                                                      3 if x == '3 - DE R$ 40.001 A R$ 70.000' else
                                                                      4 if x == '4 - DE R$ 70.001 A R$ 90.000' else
                                                                      5 if x == '5 - DE R$ 90.001 A 100.000 ' else
                                                                      6 if x == '6 - ACIMA DE R$ 100.000' else 'Nan' )

df_clientes['range_diasAtraso'] = df_clientes['range_diasAtraso'].apply( lambda x: 0 if x == 'NO PRAZO' else
                                                            1 if x == '1 - DE 1 A 5 DIAS' else
                                                            2 if x == '2 - DE 6 A 10 DIAS' else
                                                            3 if x == '3 - DE 11 A 15 DIAS' else
                                                            4 if x == '4 - DE 16 A 20 DIAS' else
                                                            5 if x == '5 - DE 20 A 25 DIAS' else
                                                            6 if x == '6 - ACIMA DE 25 DIAS' else 'Nan' )  
                                                    
df_clientes['range_idade'] = df_clientes['range_idade'].apply( lambda x: 1 if x == '1 - De 18 a 25 Anos' else
                                                            2 if x == '2 - De 26 a 40 Anos' else
                                                            3 if x == '3 - De 41 a 55 Anos' else
                                                            4 if x == '4 - De 56 A 70 Anos' else
                                                            5 if x == '5 - De 71 A 80 Anos' else
                                                            6 if x == '6 - Acima de 80 Anos' else 'Nan' )    

df_clientes['range_estadoRendaPercapita'] = df_clientes['range_estadoRendaPercapita'].apply( lambda x: 1 if x == '1 - De R$ 500 A R$ 800' else
                                                            2 if x == '2 - De R$ 801 A R$ 1.000' else
                                                            3 if x == '3 - DE R$ 1.001 A R$ 1.200' else
                                                            4 if x == '4 - DE R$ 1.201 A R$ 1.500' else
                                                            5 if x == '5 - DE R$ 1.501 A R$ 1.800' else
                                                            6 if x == '6 - ACIMA DE R$ 1.801' else 'Nan' ) 

df_clientes['range_rendaMensal'] = df_clientes['range_rendaMensal'].apply( lambda x: 1 if x == '1 - DE R$ 1.500 A R$ 2.000' else
                                                            2 if x == '2 - DE R$ 2.001 A R$ 4.000' else
                                                            3 if x == '3 - DE R$ 4.001 A R$ 6.000' else
                                                            4 if x == '4 - DE R$ 6.001 A R$ 8.000' else
                                                            5 if x == '5 - DE R$ 8.001 A R$ 9.000' else
                                                            6 if x == '6 - ACIMA DE R$ 9.000' else 'Nan' )

df_clientes['range_tempoRelacionamento'] = df_clientes['range_tempoRelacionamento'].apply( lambda x: 1 if x == '1 - DE 1 A 3 ANOS' else
                                                                              2 if x == '2 - DE 4 A 10 ANOS' else
                                                                              3 if x == '3 - DE 10 A 20 ANOS' else
                                                                              4 if x == '4 - DE 21 A 25 ANOS' else
                                                                              5 if x == '5 - DE 26 A 30 ANOS' else
                                                                              6 if x == 'ACIMA DE 30 ANOS' else 'Nan' )  

df_clientes['range_cidadeRendaPercapita'] = df_clientes['range_cidadeRendaPercapita'].apply( lambda x: 1 if x == '1 - De R$ 500 A R$ 800' else
                                                                                          2 if x == '2 - De R$ 801 A R$ 1.000' else
                                                                                          3 if x == '3 - DE R$ 1.001 A R$ 1.200' else
                                                                                          4 if x == '4 - DE R$ 1.201 A R$ 1.500' else
                                                                                          5 if x == '5 - DE R$ 1.501 A R$ 1.800' else
                                                                                          6 if x == '6 - ACIMA DE R$ 1.801'else 'Nan' )



colunas_categoricas = [coluna for coluna in df_clientes.columns if coluna not in df_clientes._get_numeric_data().columns]

In [ ]:
for coluna_categorica in colunas_categoricas:
    display(df_clientes[coluna_categorica].value_counts())

In [ ]:
# Vamos utilizar OHE para variáveis categóricas nominais
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder(sparse=False, drop='first')
df_ohe_transformadas = ohe.fit_transform(df_clientes[colunas_categoricas])
ohe.categories_

In [ ]:
ohe.get_feature_names_out()

In [ ]:
df_ohe_transformadas

In [ ]:
# Tranformando o array numpy em colunas.
df_ohe_transformadas = pd.DataFrame(data=df_ohe_transformadas, columns=ohe.get_feature_names_out(), index=df_clientes.index)
df_ohe_transformadas.head()

In [ ]:
df_clientes.shape

In [ ]:
df_ohe_transformadas.shape

In [ ]:
# Utilizando o concat para realizar um "JOIN" entre os dataframes original e com as colunas com one hot encoding
# axis=0 ele apensaria as linhas, axis=1 ele junta as colunas
df_clientes = pd.concat([df_clientes, df_ohe_transformadas], axis=1)
df_clientes.head()

In [ ]:
colunas_categoricas


In [ ]:
# Agora precisamos remover as colunas originais!
df_clientes = df_clientes.drop(colunas_categoricas, axis=1)
df_clientes.head()

In [ ]:
df_clientes.dtypes

In [ ]:
# Atualizando as colunas que ainda são categóricas
colunas_categoricas = [coluna for coluna in df_clientes.columns if coluna not in df_clientes._get_numeric_data()]
colunas_categoricas

In [ ]:
df_clientes.columns

In [ ]:
X = df_clientes.drop(['range_diasAtraso', 'status'], axis=1) # tirando a variável dependente
y = df_clientes[['status']] # extraindo a variável dependente

In [ ]:
# Variáveis preditoras (ou independentes ou, features)
X.head()

In [ ]:
# Variável dependente, ou target, ou label (ah, vcs entenderam :)
y.head()

In [ ]:
# A função que separa nossa base em treino e teste! 
# Lembrando que faremos cross validation com a base de treino
from sklearn.model_selection import train_test_split

In [ ]:
# Devolve uma tupla com 4 elementos: X de treino, X de teste, y de treino, y de teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, # preditoras 
                                                        y, # target
                                                        test_size=.2, 
                                                        random_state=42)

# Vamos ver quantas linhas ficamos com treino e teste
X_treino.shape, X_teste.shape, y_treino.shape, y_teste.shape

In [ ]:
X_treino.head()

In [ ]:
y_treino.head()

In [ ]:
X_treino.shape[0] / X.shape[0]

In [ ]:
# isnull busca quem é nulo (dados faltantes)
X_treino.isnull().sum()

In [ ]:
X_teste.isnull().sum()

In [ ]:
# Veja como Regressão Logística está no pacotão de modelos lineares!
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
# Criando o estimador, algoritmo, modelo, preditor, classificador (virge, que tanto de nome!)
# Vamos alterar o número de iterações para cálculo da regressão logística, pois no default ele enche de warnings
# que pode não ter chegado na melhor solução
regressao_logistica = LogisticRegression(max_iter=2500)

In [ ]:
type(regressao_logistica)

In [ ]:
# Vamos treinar utilizando cross validation
valores_f1_rl = cross_val_score(estimator=regressao_logistica, 
                                  X=X_treino, 
                                  y=y_treino.values.flatten(), 
                                  cv=10, # 10-fold CV
                                  scoring='accuracy') # teste com accuracy para verificar acurácia do modelo
valores_f1_rl

In [ ]:
len(valores_f1_rl)

In [ ]:
media_f1_rl = valores_f1_rl.mean()
f'accuracy: {media_f1_rl*100}'

In [ ]:
# Modelo de bagging mais famoso!
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()

# Vamos treinar utilizando cross validation (sempre!!)
valores_f1_rf = cross_val_score(estimator=random_forest, 
                                      X=X_treino, 
                                      y=y_treino.values.flatten(), 
                                      cv=10, # 
                                  scoring= 'accuracy')
valores_f1_rf

In [ ]:
media_f1_rf = valores_f1_rf.mean()
f'accuracy: {media_f1_rf*100}'

In [ ]:
import xgboost as xgb

In [ ]:
xgb_model = xgb.XGBClassifier(random_state=42, 
                              objective='binary:logistic', 
                              use_label_encoder=False, 
                              eval_metric='error')

In [ ]:
type(xgb_model)

In [ ]:
# Vamos treinar utilizando cross validation (sempre!!)
valores_f1_xgb = cross_val_score(estimator=xgb_model, 
                                      X=X_treino, 
                                      y=y_treino.values.flatten(), 
                                      cv=10, # 
                                  scoring='accuracy')
valores_f1_xgb

In [ ]:
media_f1_xgb = valores_f1_xgb.mean()

f'accuracy: {media_f1_xgb*100}'

In [ ]:
regressao_logistica.fit(X_treino, y_treino.values.flatten())

In [ ]:
regressao_logistica.coef_

In [ ]:
df_coeficientes = pd.DataFrame(regressao_logistica.coef_)
df_coeficientes.columns=regressao_logistica.feature_names_in_
df_coeficientes

## <a> Finalmente </a>

Agora que temos nosso modelo final, podemos fazer inferências dos valores do churn no teste. Percebam que nunca utilizamos o teste PARA NADA, como deve ser.

In [ ]:
# ver estimadores scikit learn
# estimador é treinado com fit
# estimador prediz com predict
predicoes_status = regressao_logistica.predict(X_teste)
predicoes_status[:10]

In [ ]:
len(predicoes_status)

In [ ]:
y_teste.head()

In [ ]:
predicoes_vs_real = pd.DataFrame({'predicao': predicoes_status.flatten(), 'real': y_teste.values.flatten()})
predicoes_vs_real.head(10)

In [ ]:
# Tudo muito bem, tudo muito bom. Mas será que uma simples média é melhor do 
# que nosso modelo? Vamos testar o r quadrado
from sklearn.metrics import f1_score

f1_score(y_true=y_teste, y_pred=predicoes_status)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true=y_teste, y_pred=predicoes_status)

In [ ]:
predicoes_vs_real['predicao'].nunique()
# ao verificar valores de predição é possível notar que o modelo definiu todos como 1, como corrigir isso?
